In [93]:
import os
import nltk
import csv
import pandas as pd
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
import re
from nltk.stem.isri import ISRIStemmer


# لفائدتها في التصنيف لاحقاStopWords استثناء بعض الكلمات من ال 

In [94]:
##Some StopWords Is Important SO Keep Them 

important=['ولا','لولا','نعم','ليس','لكن','كلا','ليست','لا']
stopword=stopwords.words('arabic')
stopword=[stopword.remove(K)for K in important if K in stopword]

# Bow Features And tfidf_features extraction

In [95]:
###Bow Features And tfidf_features extraction

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

##############################################################################################
def bow_features(X_train, X_test,ngram_range,minn,maxx):
    count_vectorizer = CountVectorizer(lowercase =False,stop_words=stopword,
                                       ngram_range=ngram_range,min_df=minn,max_df=maxx)
    X_train_Vec =count_vectorizer.fit_transform(X_train) 
    X_test_Vec = count_vectorizer.transform(X_test) 
    
    return X_train_Vec, X_test_Vec, count_vectorizer.vocabulary_, count_vectorizer
##############################################################################################
def tfidf_features(X_train, X_test,ngram_range,minn,maxx):
    tfidf_vectorizer = TfidfVectorizer(lowercase=False,stop_words=stopword,
                                       ngram_range=ngram_range,min_df=minn,max_df=maxx)
    X_train =tfidf_vectorizer.fit_transform(X_train) 
    X_test = tfidf_vectorizer.transform(X_test) 

    return X_train, X_test, tfidf_vectorizer.vocabulary_, tfidf_vectorizer

# Training The Classifier Model

In [96]:
#Training The Classifier Model

from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression, RidgeClassifier
def train_classifier(X_train, y_train):
    classifier = RidgeClassifier()
    classifier.fit(X_train, y_train)
    return classifier

#  بحيث كل ملف يمثل عنصر منهاListقراءة مجلد داتا التدريب وتخزينه في 

In [74]:
##Read Training Files From Folder And Store The Countent Of Them In List
entries = os.listdir('Without_Processing\\Training_Data')

data=[]
all=0
target=r'(?P<starter>\|\|\s*)(?P<target>((POS|NEG|pos|neg)))'
for entry in entries:
    newText=[]
    #print(entry)
    f=open('Training_Data/'+(entry),'r',encoding='utf-8')
    text=f.read()
    text=text.split('\n')
    #text=[t.split('||')for t in text]
    for t in text:
        m = re.search(target,t)
        if m:
            clas=m.group('target')
            temp=[re.sub(target,'',t),clas.upper()]
            newText.append(temp) 
        
    data.append(newText)  
    f.close()
    
print("Number Of Files "+str(len(data)))

Number Of Files 73


# تصفية التويتات الغير مصنفة 

In [97]:
##now we Have List Of List Each List Represent Data OF File
##Convert List Of List Of Tweets To List Of Tweets##
file=0
line=1
all_data=[]
all=0
for text in data :
    line=1
    file+=1
    for x in text :
        all+=1
        if len(x)!=2:
            print("file %d" %file)
            #print(" in line %d" %line)    
        elif x[1].upper().strip()=="POS" or x[1].upper().strip()=="NEG":
            all_data.append([x[0],x[1].upper().strip()])
            #print("OK")
        else:
            print("Problem with file %d" %file)
            print(" in line %d" %line)    
            print(x[0])
        line+=1
print("Number Of All Tweets "+str(len(all_data)))

Number Of All Tweets 7409


#  csvوتخزينها في ملفlistحذف التكرار من  ال

In [98]:
##Get Uniqe Tweets
uniqe_data=[]
[uniqe_data.append(x) for x in all_data if x not in uniqe_data]
outfile=open('data.csv', 'w',encoding="utf-8")
writer = csv.writer(outfile)

for t in uniqe_data:
    writer.writerow(t)
print(len(uniqe_data))
for t in uniqe_data:
    if(len(t)<2):
        print (t)
print("Number Of Uniqe Tweets "+str(len(uniqe_data)))
outfile.close()

6814
Number Of Uniqe Tweets 6814


# قراءة ملف التدريب csv

In [77]:
TrainData = pd.read_csv('data.csv',names=['Tweet', 'Target'], encoding = 'utf-8')
#TrainData.head(7)

In [99]:
TrainData['Target'].value_counts()

POS    6023
NEG    5278
Name: Target, dtype: int64

#   بحيث كل ملف يمثل عنصر منهاListقراءة مجلد داتا الاختبار وتخزينه في 

In [100]:
##Read Testing Files From Folder And Store The Countent Of Them In List

entries = os.listdir('Without_Processing\\Testing_Data')
i=1
data=[]
target=r'(?P<starter>\|\|\s*)(?P<target>((POS|NEG|pos|neg)))'
newText=[]
for entry in entries:
    newText=[]
    j=0
    #print(entry)
    f=open('Testing_Data/'+(entry),'r',encoding='utf-8')
    text=f.read()
    text=text.split('\n')
    for t in text:
        j+=1
        m = re.search(target,t)
        if m:
            clas=m.group('target')
            temp=[re.sub(target,'',t),clas.upper()]
            newText.append(temp) 
           
    data.append(newText)
    f.close()
    
    
    
print("Number Of Files "+str(len(data)))


Number Of Files 37


#  تصفية التويتات الغير مصنفة 

In [101]:
file=0
line=1
all_data=[]
for text in data :
    line=1
    file+=1
    for x in text :
        if len(x)!=2:
            print("file %d" %file)
            print(" in line %d" %line)    
        elif x[1].upper().strip()=="POS" or x[1].upper().strip()=="NEG":
            all_data.append([x[0],x[1].upper().strip()])
            #print("OK")
        else:
            print("Problem with file %d" %file)
            print(" in line %d" %line)    
            print(x[0])
        line+=1
print("Number Of All Tweets "+str(len(all_data)))


Number Of All Tweets 7409


# csvوتخزينها في ملفlistحذف التكرار من  ال

In [102]:
##Get uniqe Tweets
uniqe_data=[]
[uniqe_data.append(x) for x in all_data if x not in uniqe_data]
outfile=open('Testdata.csv', 'w',encoding="utf-8")
writer = csv.writer(outfile)

for t in uniqe_data:
    writer.writerow(t)
print(len(uniqe_data))
for t in uniqe_data:
    if(len(t)<2):
        print (t)
print("Number Of Uniqe Tweets "+str(len(uniqe_data)))
outfile.close()

6814
Number Of Uniqe Tweets 6814


# قراءة ملف داتا الاختبار csv

In [82]:
Testdata = pd.read_csv('Testdata.csv',names=['Tweet', 'Target'], encoding = 'utf-8')
#Testdata.head(7)

In [83]:
Testdata['Target'].value_counts()

POS    3435
NEG    3379
Name: Target, dtype: int64

#    Text_Cleaning_and_Normalization
حذف الروابط والصور والحروف المكررة وعلامات الترقيم
وتبديل بعض الحروف كالهمزة و... بأحرف بديلة 
حذف المنشن والريتويت Mention And Retweet
حذف علامات التشكيل

#     Text_Analysis
Steemingارجاع الكلمات للجذر




In [103]:
def normalize(text):
    text = re.sub("ة", "ه",text)
    text = re.sub("[إأآا]", "ا",text)
    text = re.sub("ى", "ي",text)
    text = re.sub("ئ", "ء",text)
    text=re.sub(r'(.)\1+', r'\1\1',text)
    return text

###############################################################################################
def Text_Cleaning_and_Normalization(text):
    URLPic=r'pic.twitter.com/[A-Za-z0-9]+'#الصور
    URLLink=r'http(s)?://(\s)*[A-Za-z]+\.[A-Za-z]+((/)?([A-Za-z0-9])+(\s)?)+'#الروابط
    punctuations_Symbols = r'[\.؟!;()&,%"''?]'#علامات الترقيم
    arabic_diacritics = r' َ  ُ ِ ّ  ً  ٌ ٍ  ْ '#علامات التشكيل
    EngNames=r'@[A-Za-z0-9_ا-ي]+'#منشن او ريتويت باللغة الانكليزية
    ArbNames=r'[0-9_ا-ي]+@'#منشن اة ريتويت باللغة العربية

    
    text=re.sub(URLLink,'',text)
    text=re.sub(URLPic,'',text)
    text=re.sub(punctuations_Symbols,'',text)
    text=re.sub(arabic_diacritics,'',text)
    text=re.sub(EngNames,'',text)
    text=re.sub(ArbNames,'',text)
    text=normalize(text)
    return text

###############################################################################################
#Text_Analysis
stop_words = set(stopwords.words('arabic'))
def Text_Analysis(text,isstem):
    if isstem:
        text=text.split()
        st = ISRIStemmer()
        text=[st.stem(w)for w in text]
        text=''.join(text)
    return text



# استبدال  الهاشتاغات
# إزالة الهاشتاغات

In [104]:
def remove_hashtag(text):
    hashtag=r'#([\w]|[0-9])+'
    text=re.sub(hashtag,'',text)
    return text
##############################################################################################
def convert_hashtag(text):
    
    #print(text)
    hashtag=r'#|_'
    text=re.sub(hashtag,' ',text)

    #print(text)
    return text

# اختبار استبدال وازالة الهاشتاغ على مثال من التويتات

In [105]:
print("Example For Removing and Converting Hashtags")
te='#سعوديات_نطلب_اسقاط_الولايه692 يجب أن تسقط الولاية، رغم أن سقوطها وحده لا يكفي يحب أن يوفر الدعم للمرأة في التعليم و العمل و تكوين أسرة '
print("remove_hashtag \n"+remove_hashtag(str(te)))
te='#سعوديات_نطلب_اسقاط_الولايه692 يجب أن تسقط الولاية، رغم أن سقوطها وحده لا يكفي يحب أن يوفر الدعم للمرأة في التعليم و العمل و تكوين أسرة '
print("convert_hashtag \n"+convert_hashtag(str(te)) ) 

Example For Removing and Converting Hashtags
remove_hashtag 
 يجب أن تسقط الولاية، رغم أن سقوطها وحده لا يكفي يحب أن يوفر الدعم للمرأة في التعليم و العمل و تكوين أسرة 
convert_hashtag 
 سعوديات نطلب اسقاط الولايه692 يجب أن تسقط الولاية، رغم أن سقوطها وحده لا يكفي يحب أن يوفر الدعم للمرأة في التعليم و العمل و تكوين أسرة 


#(tweet,Target\pos|neg)تجهيز الداتا بحيث كل عنصر يمثل ك

In [87]:
traindata = [tuple(x) for x in TrainData.values]
testdata = [tuple(x) for x in Testdata.values]

#  للكلماتsteeming تابع يقوم بتنظيف الداتا وعمل 

In [106]:
##Cleaning And Steeming
def preprocess(traindata,testdata,isstem,isclean):
    if isclean:
        train_data_norm=[(Text_Cleaning_and_Normalization(str(t[0])),t[1])for t in traindata]
        test_data_norm=[(Text_Cleaning_and_Normalization(str(t[0])),t[1])for t in testdata]
    else:
        train_data_norm=traindata
        test_data_norm=testdata

    train_data_norm=[(Text_Analysis(str(t[0]),isstem),t[1])for t in train_data_norm] 
    test_data_norm=[(Text_Analysis(str(t[0]),isstem),t[1])for t in test_data_norm]
    return train_data_norm,test_data_norm



# X وYفصل الداتا ل

In [107]:
#Splite Data to Training And Testing
def splite_data(traindata,testdata,isstem,isclean):
    train_data_norm,test_data_norm=preprocess(traindata,testdata,isstem,isclean)
    training_set_size = int(len(train_data_norm))
    testing_set_size = int(len(test_data_norm))

    X_train = [example[0] for example in train_data_norm]
    y_train = [example[1] for example in train_data_norm]

    X_test = [example[0] for example in test_data_norm]
    y_test = [example[1] for example in test_data_norm]
    return X_train,y_train,X_test,y_test


# تابع يقوم ب
## Get Features Of Data With bow And tfidf
## Then Training And Testing The Model 
## ComputeThe Accuracy
#### min_df , max_dfو البارامترين N_Gramنقوم بتمرير داتا التدريب وداتا الاختبار وعدد ال 
####  وتدريب المودل features نقوم بإيجاد ال
#### accuracyنقوم باختبار المودل وحساب وطباعة ال 

In [108]:
##Get Features Of Data With bow And tfidf
##Then Training And Testing The Model 
## ComputeThe Accuracy 
def Get_Result(X_train,X_test,y_train,y_test,ngrams,minn,maxx):
    X_train_bag, X_test_bag, bow_vocab, bow_vec= bow_features(X_train, X_test,ngrams,minn,maxx)
    X_train_tfidf, X_test_tfidf, tfidf_vocab, tfidf_vec= tfidf_features(X_train, X_test,ngrams,minn,maxx)
    #############################################################################################
    classifier_bag = train_classifier(X_train_bag, y_train)
    classifier_tfidf = train_classifier(X_train_tfidf, y_train)
    #############################################################################################
    y_test_predicted_labels_bag = classifier_bag.predict(X_test_bag)
    y_test_predicted_scores_bag = classifier_bag.decision_function(X_test_bag)

    y_test_predicted_labels_tfidf = classifier_tfidf.predict(X_test_tfidf)
    y_test_predicted_scores_tfidf = classifier_tfidf.decision_function(X_test_tfidf)
    #############################################################################################
    #UniGramm Without Steeming
    from sklearn.metrics import accuracy_score

    print('Bag-of-words Accuracy: '+ str(accuracy_score(y_test, y_test_predicted_labels_bag)))
    print('Tfidf Accuracy: ' + str(accuracy_score(y_test, y_test_predicted_labels_tfidf)))
    
    return bow_vec,tfidf_vec,classifier_bag,classifier_tfidf

### 1) تجريب المودل  Bigram##Without Cleaning The Data

In [91]:
X_train,Y_train,X_test,Y_test=splite_data(traindata,testdata,False,False)

In [109]:
print('------------------------------------------------------')
print("Bigram##Without Cleaning The Data ")
bow_vec,tfidf_vec,classifier_bag,classifier_tfidf=Get_Result(X_train,X_test,Y_train,Y_test,(1,2),1,0.09)
print('------------------------------------------------------')

------------------------------------------------------
Bigram##Without Cleaning The Data 
Bag-of-words Accuracy: 0.8095098326973877
Tfidf Accuracy: 0.8219841502788376
------------------------------------------------------


## 2) تجريب المودل  Unigram##Without Cleaning The Data

In [110]:
print('------------------------------------------------------')
print("Unigram##Without Cleaning The Data ")
bow_vec,tfidf_vec,classifier_bag,classifier_tfidf=Get_Result(X_train,X_test,Y_train,Y_test,(1,1),1,0.09)
print('------------------------------------------------------')
#Bigram Give Higher Accuracy

------------------------------------------------------
Unigram##Without Cleaning The Data 
Bag-of-words Accuracy: 0.7983563252127972
Tfidf Accuracy: 0.8168476665688289
------------------------------------------------------


## 3) تجريب المودل Bigram#With Cleaning The Data 

In [111]:
print('------------------------------------------------------')
X_train,Y_train,X_test,Y_test=splite_data(traindata,testdata,False,True)
print("#Bigram#With Cleaning The Data ")
bow_vec,tfidf_vec,classifier_bag,classifier_tfidf=Get_Result(X_train,X_test,Y_train,Y_test,(1,2),1,0.09)
print('------------------------------------------------------')

------------------------------------------------------
#Bigram#With Cleaning The Data 
Bag-of-words Accuracy: 0.8128852362782507
Tfidf Accuracy: 0.82169063692398
------------------------------------------------------


## 4) تجريب المودلBigram##With Word Steeming

In [27]:
print('------------------------------------------------------')
X_train,Y_train,X_test,Y_test=splite_data(traindata,testdata,True,True)
print("Bi##With Word Steeming")
bow_vec,tfidf_vec,classifier_bag,classifier_tfidf=Get_Result(X_train,X_test,Y_train,Y_test,(1,2),1,0.09)
print('----------------------------------------------------------------')
##Steeming Word Give Less Accuracy 

------------------------------------------------------
Bi##With Word Steeming
Bag-of-words Accuracy: 0.7001761080129146
Tfidf Accuracy: 0.6916642207220428
----------------------------------------------------------------


# 5/Removing Hashtags

In [112]:
###Removing Hashtags fromData
traindata_Copy=traindata.copy()
testdata_Copy=testdata.copy()
print(traindata_Copy[0])
traindata_Copy=[(remove_hashtag(str(t[0])),t[1])for t in traindata_Copy]
testdata_Copy=[(remove_hashtag(str(t[0])),t[1])for t in testdata_Copy]
print(traindata_Copy[0])

('#سعوديات_نطلب_اسقاط_الولايه692 يجب أن تسقط الولاية، رغم أن سقوطها وحده لا يكفي يحب أن يوفر الدعم للمرأة في التعليم و العمل و تكوين أسرة ', 'POS')
(' يجب أن تسقط الولاية، رغم أن سقوطها وحده لا يكفي يحب أن يوفر الدعم للمرأة في التعليم و العمل و تكوين أسرة ', 'POS')


## 5) With Removing Hashtags & Without Cleaning The Data  تجريب المودل 

In [113]:
print('------------------------------------------------------')
X_train,Y_train,X_test,Y_test=splite_data(traindata_Copy,testdata_Copy,True,True)
print("With Removing Hashtags & Without Cleaning The Data  ")
bow_vec,tfidf_vec,classifier_bag,classifier_tfidf=Get_Result(X_train,X_test,Y_train,Y_test,(1,2),1,0.09)
print('------------------------------------------------------')

------------------------------------------------------
With Removing Hashtags & Without Cleaning The Data  
Bag-of-words Accuracy: 0.6721455826240094
Tfidf Accuracy: 0.6725858526562959
------------------------------------------------------


# 6) Bigram##With Removing Hashtags & With Cleaning The Dataتجريب المودل 

In [114]:
print('------------------------------------------------------')
X_train,Y_train,X_test,Y_test=splite_data(traindata_Copy,testdata_Copy,False,True)
print("Bigram##With Removing Hashtags & With Cleaning The Data ")
bow_vec,tfidf_vec,classifier_bag,classifier_tfidf=Get_Result(X_train,X_test,Y_train,Y_test,(1,2),1,0.09)
print('------------------------------------------------------')

##Model With HAshtags Give Higher Accuracy Than Removing Hashtags 

------------------------------------------------------
Bigram##With Removing Hashtags & With Cleaning The Data 
Bag-of-words Accuracy: 0.8005576753742295
Tfidf Accuracy: 0.81171118285882
------------------------------------------------------


## 7)  With Removing Hashtags & With Word Steemingتجريب المودل

In [36]:
print('------------------------------------------------------')
X_train,Y_train,X_test,Y_test=splite_data(traindata_Copy,testdata_Copy,True,True)
print("With Removing Hashtags & With Word Steeming")
bow_vec,tfidf_vec,classifier_bag,classifier_tfidf=Get_Result(X_train,X_test,Y_train,Y_test,(1,2),1,0.09
                                                            
                                                            )
print('------------------------------------------------------')
#Steeming Word Give Bad Accuracy

------------------------------------------------------
With Removing Hashtags & With Word Steeming
Bag-of-words Accuracy: 0.6721455826240094
Tfidf Accuracy: 0.6725858526562959
------------------------------------------------------


# 6\Converting Hashtags To Separate  Words 
# تحويل الهاشتغات الى كلمات منفصلة

In [115]:
###Converting Hashtags
traindata_Copy=traindata.copy()
testdata_Copy=testdata.copy()
print(colored("Example Before Converting",'green'))
print(traindata_Copy[0])
traindata=[(convert_hashtag(str(t[0])),t[1])for t in traindata_Copy]
testdata=[(convert_hashtag(str(t[0])),t[1])for t in testdata_Copy]
print(colored("Example After Converting",'green'))
print(traindata[0])

Example Before Converting
('#سعوديات_نطلب_اسقاط_الولايه692 يجب أن تسقط الولاية، رغم أن سقوطها وحده لا يكفي يحب أن يوفر الدعم للمرأة في التعليم و العمل و تكوين أسرة ', 'POS')
Example After Converting
(' سعوديات نطلب اسقاط الولايه692 يجب أن تسقط الولاية، رغم أن سقوطها وحده لا يكفي يحب أن يوفر الدعم للمرأة في التعليم و العمل و تكوين أسرة ', 'POS')


## 7)  تجريب المودل 
## Bigarm##With Convering Hashtags & Without Cleaning The Data 

In [116]:
print('------------------------------------------------------')
X_train,Y_train,X_test,Y_test=splite_data(traindata_Copy,testdata_Copy,True,True)
print("Bigarm##With Convering Hashtags & Without Cleaning The Data  ")
Get_Result(X_train,X_test,Y_train,Y_test,(1,2),1,0.09)
print('------------------------------------------------------')


------------------------------------------------------
Bigarm##With Convering Hashtags & Without Cleaning The Data  
Bag-of-words Accuracy: 0.7001761080129146
Tfidf Accuracy: 0.691517464044614
------------------------------------------------------


# The Best Accuracy
##  بعد تحويل الهاشتاغات وتنظيف الداتا من علامات الترقيم والاحرف المكررة والروابط والصور وحذف المنشن والريتويت 
## Bigramباستخدام 

In [117]:
print(colored("The Best Accuracy",'blue'))
print("With Cleaning Data , Converting Hashtags and Using Bigram")

print('------------------------------------------------------')
X_train,Y_train,X_test,Y_test=splite_data(traindata_Copy,testdata_Copy,False,True)
print("Bigram##With Converting Hashtags & With Cleaning The Data ")
Get_Result(X_train,X_test,Y_train,Y_test,(1,2),1,0.09)
print('------------------------------------------------------')

The Best Accuracy
With Cleaning Data , Converting Hashtags and Using Bigram
------------------------------------------------------
Bigram##With Converting Hashtags & With Cleaning The Data 
Bag-of-words Accuracy: 0.8130319929556795
Tfidf Accuracy: 0.8218373936014088
------------------------------------------------------


## تدريب المودل Unigram##With Converting Hashtags & With Cleaning The Data 

In [118]:
print('------------------------------------------------------')
X_train,Y_train,X_test,Y_test=splite_data(traindata_Copy,testdata_Copy,False,True)
print("Unigram##With Converting Hashtags & With Cleaning The Data ")
Get_Result(X_train,X_test,Y_train,Y_test,(1,1),1,0.09)
print('------------------------------------------------------')


------------------------------------------------------
Unigram##With Converting Hashtags & With Cleaning The Data 
Bag-of-words Accuracy: 0.8008511887290872
Tfidf Accuracy: 0.8156736131493983
------------------------------------------------------


## The Best Accuracy
##  بعد تحويل الهاشتاغات وتنظيف الداتا من علامات الترقيم والاحرف المكررة والروابط والصور وحذف المنشن والريتويت 
## treegramباستخدام  

In [119]:
print('------------------------------------------------------')
X_train,Y_train,X_test,Y_test=splite_data(traindata_Copy,testdata_Copy,False,True)
print("Treegram##With Converting Hashtags & With Cleaning The Data ")
Get_Result(X_train,X_test,Y_train,Y_test,(1,3),1,0.09)
print('------------------------------------------------------')

------------------------------------------------------
Treegram##With Converting Hashtags & With Cleaning The Data 
Bag-of-words Accuracy: 0.8140592896976813
Tfidf Accuracy: 0.8230114470208394
------------------------------------------------------


## تجريب المودل
## Bigram##With Converting Hashtags & With Word Steeming

In [54]:
print('------------------------------------------------------')
X_train,Y_train,X_test,Y_test=splite_data(traindata_Copy,testdata_Copy,True,True)
print("Bi##With Converting Hashtags & With Word Steeming")
Get_Result(X_train,X_test,Y_train,Y_test,(1,2),1,0.09)
print('------------------------------------------------------')

------------------------------------------------------
Bi##With Converting Hashtags & With Word Steeming
Bag-of-words Accuracy: 0.6758144995597299
Tfidf Accuracy: 0.6758144995597299
------------------------------------------------------


# 3\ Get Input From User 

In [53]:
tweet=input()
tweet=Text_Cleaning_and_Normalization(tweet)
tweetfiture=tfidf_vec.transform([tweet])
classifier_tfidf.predict(tweetfiture)[0]


مع اسقاط الولاية


'POS'

## تحميل داتا اضافية محايدة لعمل Multi Classification 

In [81]:
import GetOldTweets3 
import got3 as got

In [ ]:
#Download Data And Save It In File 
tweetCriteria = GetOldTweets3.manager.TweetCriteria()\
                                     .setQuerySearch('مباراة')\
                                     .setNear("Saudi Arabia")\
                                     .setSince("2017-11-16")\
                                     .setUntil("2019-11-18")\
                                     .setMaxTweets(1000)\

tweet = got.manager.TweetManager.getTweets(tweetCriteria)
file = open("Tweets.txt",'w',encoding='utf-8') 
for t in tweet:
    file.write(t.text+ "\n")


# (tweet,target\'nothing')قراءة الداتا الإضافية من الملف وتصنيفها ك
### لإعادة تقسيمها بشكل متوازنlist دمج الداتا في 


In [52]:
f=open('Tweets.txt','r',encoding='utf-8')
text=f.read()
text=text.split('\n')
additional_data=[(t,'Nothing')for t in text]
[traindata.append(t)for t in testdata if t not in traindata]
[traindata.append(t)for t in additional_data]

print(len(traindata))

### تقسيم الداتا بشكل متوازن لداتا تدريب وداتا اختبار 

In [63]:
training_set_size = int(len(traindata)*0.70)
X = [example[0] for example in traindata]
y = [example[1] for example in traindata]

X_train = X[:training_set_size]
y_train = y[:training_set_size]

X_test = X[training_set_size:]
y_test = y[training_set_size:]

print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))



11582
11582
4964
4964


### إعادة تدريب المودل كمسألة Multi Class Classification

In [ ]:
bow_vec,tfidf_vec,classifier_bag,classifier_tfidf=Get_Result(X_train,X_test,Y_train,Y_test,(1,2),1,0.09)